In [1]:
from IPython.core.display import Markdown, display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

import traceback
import numpy as np
import scipy.stats as ss
import yaml
import itertools
import sys
import os
import subprocess
from os import path
import pickle
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

from sklearn.linear_model import LinearRegression, TheilSenRegressor, RANSACRegressor, HuberRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import make_pipeline

def printmd(*args):
    display(Markdown(' '.join(map(str, args))))

In [2]:
#!{"ros2 run local_planning_performance_modelling compute_metrics -s"}

In [3]:
df = pd.read_csv(path.expanduser("~/ds/performance_modelling/output/test_local_planning/results.csv"))
with open(path.expanduser("~/ds/performance_modelling/output/test_local_planning/results_info.yaml")) as results_info_file:
    results_info = yaml.safe_load(results_info_file)

df = df[df.amcl_alpha_factor.notna()]

# turn odometry_error into alpha_1..4
results_info['run_parameter_names'] += ['alpha_1', 'alpha_2', 'alpha_3', 'alpha_4']
results_info['run_parameter_names'].remove('odometry_error')
for i in range(0, 4):
    df[f'alpha_{i+1}'] = df['odometry_error'].apply(lambda x: eval(x)[i])
del df['odometry_error']

# df.loc[df.robot_model == 'turtlebot3_waffle_performance_modelling', 'robot_model'] = 'turtle'
# df.loc[df.robot_model == 'hunter2', 'robot_model'] = 'hunter'

run_parameters = [c for c in list(df.columns) if c in results_info['run_parameter_names']]
metrics_versions = [c for c in list(df.columns) if '_version' in c]
everything_else = ['run_id', 'session_id', 'run_number']
metrics = [c for c in df.columns if c not in metrics_versions + run_parameters + everything_else]
metrics_and_versions = [c for c in list(df.columns) if '_version' in c or c in metrics]

cpu_time_metrics = [c for c in metrics if 'cpu_time' in c]
max_memory_metrics = [c for c in metrics if 'max_memory' in c]
odometry_metrics = [c for c in metrics if 'odometry_error_alpha_' in c]
odometry_mean_metrics = [c for c in metrics if 'odometry_error_alpha_' in c and '_mean' in c]
odometry_std_metrics = [c for c in metrics if 'odometry_error_alpha_' in c and '_std' in c]
odometry_parameters = [f'alpha_{i}' for i in range(1, 5)]
localization_update_metrics = [c for c in metrics if 'localization_update_' in c]
localization_update_mean_metrics = [c for c in metrics if 'localization_update_' in c and '_mean' in c]
localization_update_translation_mean_metrics = [c for c in metrics if 'localization_update_' in c and '_translation_' in c and '_mean' in c]
navigation_metrics = [c for c in metrics if c in ['trajectory_length', 'execution_time', 'success_rate']]

# add useful parameters
df['session_id'] =  df['run_id'].apply(lambda x:  x.split('_')[1]+'_'+x.split('_')[2]+'_'+x.split('_')[3])
df['run_number'] =  df['run_id'].apply(lambda x:  int(x.split('_')[5]))
df[max_memory_metrics] = df[max_memory_metrics]/1024**2
df.max_steering_angle_deg = df.max_steering_angle_deg.fillna(90)

printmd("## Number of Runs")
printmd(f"    {len(df.run_id.unique())}")

printmd("## Run Parameters")
for name in run_parameters:
    values = list(df[name].unique())
    printmd(f"    {name:<70}", sorted(values))

printmd("## Metrics")
for name in metrics_and_versions:
    if name in metrics_versions:
        if len(df[name].unique()) == 1:
            printmd(f"    {name:<70} {sorted(df[name].unique())}")
        else:
            printmd(f"<code><font style='background-color:yellow;font-family:monospace'>{name:<70}{sorted(df[name].unique())} </font></code> ⚠️")
    else:
        printmd(f"    {name:<70} min: {df[name].min(skipna=True):10.4g} {'avg':>15}: {df[name].mean(skipna=True):10.4g} {'max':>15}: {df[name].max(skipna=True):10.4g}]")

## Number of Runs

    1837

## Run Parameters

    amcl_alpha_factor                                                      [0.0, 0.5, 0.75, 1.0, 1.5, 2.0]

    environment_name                                                       ['7A-2', 'airlab', 'fr079', 'intel', 'mexico', 'office_b']

...

    localization_update_normalized_relative_rotation_error_std             min:          0             avg:     0.7821             max:      43.93]

    localization_update_error_version                                      [1]

    localization_update_rate_mean                                          min:     0.3642             avg:      1.157             max:       37.4]

    localization_update_rate_std                                           min:          0             avg:     0.6343             max:      14.34]

    localization_update_rate_version                                       [1]

In [4]:
run_parameters_list_path = path.expanduser("~/w/ros2_ws/src/local_planning_performance_modelling/config/direct_benchmark_configurations/run_parameters_list.yaml")
g = df[(df.robot_model=='turtlebot3_waffle_performance_modelling')].groupby(run_parameters)[
    'localization_update_rate_mean', 
    'localization_update_absolute_translation_error_mean', 
    'localization_update_absolute_rotation_error_mean',
    'localization_update_normalized_relative_translation_error_mean',
    'localization_update_normalized_relative_rotation_error_mean'
].mean()

print("len(g):", len(g), "len(g.isnan):", len(g[g.localization_update_rate_mean.isna()]))
run_parameters_list = list()
for index, row in g.iterrows():
    run_parameters_dict = dict(zip(run_parameters, list(index)))
    run_parameters_dict['odometry_error'] = [run_parameters_dict['alpha_1'], run_parameters_dict['alpha_2'], run_parameters_dict['alpha_3'], run_parameters_dict['alpha_4']]
    del run_parameters_dict['alpha_1']
    del run_parameters_dict['alpha_2']
    del run_parameters_dict['alpha_3']
    del run_parameters_dict['alpha_4']
    del run_parameters_dict['amcl_alpha_factor']
    
    run_parameters_dict['localization_node'] = 'localization_generator'
    run_parameters_dict['localization_generator_update_rate'] = float(row['localization_update_rate_mean'])
    run_parameters_dict['localization_generator_translation_error'] = float(row['localization_update_absolute_translation_error_mean'])
    run_parameters_dict['localization_generator_rotation_error'] = float(row['localization_update_absolute_rotation_error_mean'])
    run_parameters_dict['localization_generator_normalized_relative_translation_error'] = float(row['localization_update_normalized_relative_translation_error_mean'])
    run_parameters_dict['localization_generator_normalized_relative_rotation_error'] = float(row['localization_update_normalized_relative_rotation_error_mean'])
    
    run_parameters_list.append(run_parameters_dict)

with open(run_parameters_list_path, 'w') as f:
    yaml.dump(run_parameters_list, f, default_flow_style=False)
    
with open(run_parameters_list_path) as f:
    for i in yaml.safe_load(f):
        for k, v in i.items():
            print(f"{k:<50}: {v}")
        print()

len(g): 1226 len(g.isnan): 121
environment_name                                  : 7A-2
global_planner_node                               : navfn
local_planner_node                                : dwb
localization_generator_normalized_relative_rotation_error: 0.008373873558831912
localization_generator_normalized_relative_translation_error: 0.02022037189589289
localization_generator_rotation_error             : 0.0049092986703593725
localization_generator_translation_error          : 0.05668746657922405
localization_generator_update_rate                : 1.0568395061728395
localization_node                                 : localization_generator
max_steering_angle_deg                            : 90.0
odometry_error                                    : [0.0, 0.0, 0.0, 0.0]
robot_model                                       : turtlebot3_waffle_performance_modelling
run_index                                         : 0

environment_name                                  : 7A-2
global_pla

localization_generator_normalized_relative_rotation_error: 0.020300722883609918
localization_generator_normalized_relative_translation_error: 0.03450022863875503
localization_generator_rotation_error             : 0.008055059343476783
localization_generator_translation_error          : 0.0572238378937821
localization_generator_update_rate                : 0.6794244372990353
localization_node                                 : localization_generator
max_steering_angle_deg                            : 90.0
odometry_error                                    : [0.02, 0.0, 0.02, 0.0]
robot_model                                       : turtlebot3_waffle_performance_modelling
run_index                                         : 1

environment_name                                  : mexico
global_planner_node                               : navfn
local_planner_node                                : rpp
localization_generator_normalized_relative_rotation_error: 0.04484842196842025
localization_gene

localization_node                                 : localization_generator
max_steering_angle_deg                            : 90.0
odometry_error                                    : [0.02, 0.0, 0.02, 0.0]
robot_model                                       : turtlebot3_waffle_performance_modelling
run_index                                         : 0

environment_name                                  : intel
global_planner_node                               : navfn
local_planner_node                                : dwb
localization_generator_normalized_relative_rotation_error: 0.4237262982691791
localization_generator_normalized_relative_translation_error: 0.0673430534490263
localization_generator_rotation_error             : 0.04393716076938206
localization_generator_translation_error          : 0.10264612438801717
localization_generator_update_rate                : 0.8594594594594596
localization_node                                 : localization_generator
max_steering_angle_deg   